# HERON Paths

In [3]:
import os
import sys
import operator
import numpy as np
import subprocess
import shutil
import pandas as pd

sys.path.append('..')

from ExternalDispatchManager import ExternalDispatchRunner

raven = "~/projects/raven/raven_framework"
heron = "~/projects/HERON/heron"
mra   = "~/projects/multiresolutionanalysis"

RAVEN_LOC = os.path.expanduser(raven)
RAVEN_DIR = os.path.abspath( os.path.join(RAVEN_LOC, '..') )
sys.path.append(RAVEN_LOC)
sys.path.append(RAVEN_DIR)

HERON_LOC = os.path.expanduser(heron)
HERON_DIR = os.path.abspath( os.path.join(HERON_LOC, '..') )
sys.path.append(HERON_LOC)
sys.path.append(HERON_DIR)

MRA_LOC = os.path.expanduser(mra)
sys.path.append(os.path.join(MRA_LOC, '..'))
sys.path.append(MRA_LOC)

from src.DispatchManager import DispatchRunner
from ravenframework import ROMExternal
from ravenframework.utils import xmlUtils

/home/sotogj/.conda/envs/raven_libraries/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sotogj/.conda/envs/raven_libraries/lib/python3.10/site-packages/ray/_private/parameter.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/home/sotogj/.conda/envs/raven_libraries/lib/python3.10/site-packages/pkg_resources/__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
2024-09-25 12:01:04,957	INFO util.py:159 -- Missing pack

In [4]:
from HERON.src.dispatch.MRA_testing.utils.process_files import run_heron, get_heron_lib
from multiresolutionanalysis.notebooks.dispatch.utils.parsing_results import fix_path, check_dir


## HERON XML

Run HERON on the given case and get `heron.lib`

In [5]:
CASE_INFO = {}
CASE_INFO['ROM']     = 'fGOLD'
CASE_INFO['CASE']    = 'Case1'
CASE_INFO['METRIC']  = 'NPV_C'
CASE_INFO['SAMPLES'] = 1

MRA_scripts_dir = check_dir( os.path.join(mra, 'output/DISPATCH_COLLECT') )
MRA_test_dir    = check_dir( os.path.join(HERON_DIR, 'src/dispatch/MRA_testing/lib_files') )

HERON_lib = get_heron_lib(HERON_LOC,
                          MRA_scripts_dir,
                          MRA_test_dir,
                          CASE_INFO,
                          rerun=True,
                          removeCapex=True)

XML does not exist, copying over from /home/sotogj/projects/HERON/src/dispatch/MRA_testing/lib_files
Running HERON on /home/sotogj/projects/HERON/src/dispatch/MRA_testing/lib_files/HERON_Case1__debugNoCap_from_NPV_C.xml


## Dispatch - Set Up
Load up the `heron.lib` file and make necessary modifications...

In [6]:
dispatch_runner = ExternalDispatchRunner()
dispatch_runner.load_heron_lib(HERON_lib)

/home/sotogj/.conda/envs/raven_libraries/lib/python3.10/site-packages/pyutilib/misc/import_file.py:11: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


From the given `heron.lib` file, get all sources and load them as needed. So far, using CSV but will expand to ARMA (and functions maybe?)

In [7]:
macro_name = dispatch_runner._case.get_year_name()
time_name  = dispatch_runner._case.get_time_name()

CSV  = {}
ARMA = {}
FUNC = {}

for i,source in enumerate(dispatch_runner._sources):
    match source._type:
        case 'CSV':
            CSV[i] = {}
            CSV[i]['targets']   = source._var_names
            CSV[i]['file']      = source._target_file
            CSV[i]['dataframe'] = pd.read_csv(CSV[i]['file'])
            CSV[i]['_indexMap'] = [{target: [time_name, macro_name, '_ROM_Cluster']
                                    for target in CSV[i]['targets']}]
        case 'ARMA':
            ARMA[i] = {}
            ARMA[i]['targets'] = source._var_names
            ARMA[i]['file'] = source._target_file

### Mimicking an "outer" to get raven_dict
To run HERON dispatch, we need a `raven_dict` which is something that would get populated in the Outer optimization.

We need to get things like component capacities and the actual synthetic/static histories.

In [8]:
raven_dict = {}
raven_dict['scaling'] = np.array([1])

Component capacities handled here (assuming for most of them, we will pull the `debug_value` which was earlier extracted from full optimization runs).

In [9]:
# component capacities
for comp in dispatch_runner._components:
    print(comp.name)

    # Producer/Storage/Demands
    interaction = comp.get_interaction()
    crossrefs   = comp.get_crossrefs() # dict of stuff needed

    # every capacity will come from a VP
    if interaction in crossrefs:
        val = None
        for key, vp in crossrefs[interaction].items():
            VP = vp._vp
            debug_value = VP._debug_value if hasattr(VP,'_debug_value') else None
            if debug_value:
                val = np.array([debug_value])
            else:
                if VP.type == 'FixedValue':
                    val = np.array([VP._parametric])
                elif VP.type in ['OptBounds','SweepValues']:
                    val = np.array([VP._parametric[0]])
                else:
                    continue

            raven_dict[f'{comp.name}{key}'] = val


npp
market_electricity
source_electricity
sink_electricity


Now handling the synthetic or static histories. Assuming no mix-and-match between CSV and ARMA.

In [10]:
SOURCES = [CSV, ARMA, FUNC]

for SOURCE in SOURCES:
    for i,ROM in SOURCE.items():
        DF = ROM['dataframe']
        raven_dict['_indexMap']    = ROM['_indexMap']
        raven_dict['_ROM_Cluster'] = np.sort(np.unique(DF['_ROM_Cluster'].to_list()))
        raven_dict[macro_name]     = np.sort(np.unique(DF[macro_name].to_list()))
        raven_dict[time_name]      = np.sort(np.unique(DF[time_name].to_list()))

        for target in ROM['targets']:
            raven_dict[target] = np.zeros([len(raven_dict[time_name]),
                                           len(raven_dict[macro_name]),
                                           len(raven_dict['_ROM_Cluster'])])
            for c,cluster in enumerate(raven_dict['_ROM_Cluster']):
                for m,macro in enumerate(raven_dict[macro_name]):
                    tmp_DF = DF.loc[(DF[macro_name]==macro) & (DF['_ROM_Cluster']==cluster)]

                    raven_dict[target][:,m,c] = np.array(tmp_DF[target].to_list())



In [11]:
raven_vars = dispatch_runner.extract_variables_no_raven(raven_dict)
dispatch, metrics, tot_activity = dispatch_runner.run(raven_vars)

RUNNING HERON DISPATCH MANAGER
Start: 0 End: 8760
DEBUGG using solver: ipopt
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 6.2121782302856445 s
Start: 0 End: 8760
DEBUGG using solver: ipopt
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 6.397555351257324 s
Start: 0 End: 8760
DEBUGG using solver: ipopt
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 6.137526273727417 s
Start: 0 End: 8760
DEBUGG using solver: ipopt
DEBUGG solve attempt 1 ...:
DEBUGG ... solve was successful!
DEBUGG ... validating ...
DEBUGG Solve successful and no validation concerns raised.
DEBUGG solve time: 6.914992570877075 s
Start: 0 End: 8760
DEBUGG using solver: ipop

/home/sotogj/projects/TEAL/src/main.py:450: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  projCf[decomissionMask] += lifeCf[-1] * taxMult * np.power(inflRate, -1*years[decomissionMask])


In [30]:
import pickle
import time as time_mod
collection_file = fix_path( os.path.join(MRA_test_dir,
                                         f'collect__{time_mod.strftime("%Y_%m_%d", time_mod.localtime())}.pk') )

collection = {}
collection['raven_dict'] = raven_dict
collection['case_info']  = CASE_INFO
collection['dispatch_runner'] = dispatch_runner
collection['dispatch'] = dispatch
collection['metrics'] = metrics
collection['tot_activity'] = tot_activity

with open(collection_file, 'wb') as fp:
    pickle.dump(collection, fp)